# Minimiser le nombre d'ouvertures d'usines et le coût de transport.

Source : John Hedengren
https://stackoverflow.com/questions/62439295/gekko-optimization


In [1]:
from gekko import GEKKO
import numpy as np
import pandas as pd
import math

m = GEKKO(remote=False) #So that it solves the problem locally
m.options.SOLVER = 1 #MILP

varx = m.Array(m.Var,(5,12),lb=0,integer=True)
vary = m.Array(m.Var,5,lb=0,ub=1,integer=True)

dists = np.array([[0 , 93, 69, 98, 55, 37, 128, 95, 62, 42, 82, 34], #Boston
                 [37, 65, 33, 103, 20, 0, 137, 113, 48, 72, 79, 41], #Nashua
                 [42, 106, 105, 73, 92, 72, 94, 57, 104, 0, 68, 38], #Providence
                 [82, 59, 101, 27, 93, 79, 63, 57, 127, 68, 0,  47], #Springfield
                 [34, 68, 72, 66, 60, 41, 98, 71, 85, 38, 47,   0]]) #Worcester

max_dist = 60 #Max average distance (in miles)
min_pct = 0.8 #Min percent of demand within 50 miles

#Creates a 0s and 1s array. If the distance to a costumer 
#is less than 50, then the matrix element is 1, it is zero otherwise
aij = [[1 if dists[i,j]<=50 else 0 for j in range(12)] for i in range(5)]

dem_consts = np.array([425, 12, 43, 125, 110, 86, 129, 28, 66, 320, 220, 182])
fixd_cost = 10000
sum1 = np.sum(np.multiply(varx, dists))
sum2 = np.sum(vary*fixd_cost)
z = sum1 + sum2 
tot_dem = np.sum(dem_consts)
M = tot_dem
m.Minimize(z)

for j in range(12):
    m.Equation(m.sum(varx[:,j]) >= dem_consts[j]) #Demand constraints

for i in range(5):
    m.Equation(m.sum(varx[i,:]) <= 2000) #Capacity constraints
    m.Equation(m.sum(varx[i,:]) <= M*vary[i]) #Enforces 0 or 1 value

m.Equation(m.sum(vary) >= 1)


di_sum = np.sum(np.multiply(varx, dists))
di_sumw = di_sum/ tot_dem
m.Equation(di_sumw <= max_dist) #Average (demand) weighted distance from DC to customer

a_sum = np.sum(np.multiply(varx, aij)) 
a_sumw = m.Intermediate(a_sum/tot_dem)
m.Equation(a_sumw >= min_pct) #Percent of demand that is within 50 miles


m.solver_options = ['minlp_gap_tol 1.0e-2',\
                    'minlp_maximum_iterations 10000',\
                    'minlp_max_iter_with_int_sol 5000']
m.solve(disp = True)


p1 = np.zeros((5, 12))

for i in range (5):
    for j in range (12):
        p1[i][j] = varx[i][j].value[0]
p1t = np.transpose(p1)

p2 = np.zeros(5) 
for i in range(5):
    p2[i] = vary[i].value[0] 

mad1 = np.sum(np.multiply(p1, dists)) 
mad2 = mad1/tot_dem
mad3 = mad1/np.sum(p1)
mpi1 = np.sum(np.multiply(p1, aij)) 
mpi2 = mpi1/tot_dem
mpi3 = mpi1/np.sum(p1)

tot1 = np.sum(np.multiply(p1, dists))
tot2 = np.sum(p2)*fixd_cost
tot = tot1 + tot2 

print(p1)
print(p2)
print('The minimum cost is:' +str(tot))
print('Average customer distance (mad2):' +str(mad2))
print('Average customer distance (mad3):' +str(mad3))
print('Percent of customers <= 50 miles (mpi2):' +str(mpi2))
print('Percent of customers <= 50 miles (mpi3):' +str(mpi3))

dc = np.array(['Boston', 'Nashua', 'Providence', 'Springfield', 'Worcester'])
cities = ['Boston', 'Brattleboro', 'Concord', 'Hartford', 'Manchester', 'Nashua',
          'New Haven', 'New London', 'Portsmouth', 'Providence', 'Springfield', 'Worcester']
data = {cities[0]: p1t[0], cities[1]: p1t[1], cities[2]: p1t[2], cities[3]: p1t[3],
       cities[4]: p1t[4], cities[5]: p1t[5], cities[6]: p1t[6], cities[7]: p1t[7],
       cities[8]: p1t[8], cities[9]: p1t[9], cities[10]: p1t[10], cities[11]: p1t[11]}

df = pd.DataFrame(data, index = dc)
df

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  90
   Intermediates:  1
   Connections  :  0
   Equations    :  27
   Residuals    :  26
 
 Number of state variables:    90
 Number of total equations: -  25
 Number of slack variables: -  25
 ---------------------------------------
 Degrees of freedom       :    40
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    3 Obj:  3.06E+04 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    0 Dpth:    1 Lvs:    2 Obj:  3.06E+04 Gap:       NaN
Iter:     3 I:  0 Tm:      0.00 NLPi:    3 Dpth:    1 Lvs:    4 Obj:  3.76E+04 Gap: 

,Boston,Brattleboro,Concord,Hartford,Manchester,Nashua,New Haven,New London,Portsmouth,Providence,Springfield,Worcester
Boston,1102.0,0.0,43.0,0.0,110.0,86.0,0.0,0.0,66.0,0.0,0.0,182.0
Nashua,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Providence,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,495.0,0.0,0.0
Springfield,0.0,12.0,0.0,125.0,0.0,0.0,129.0,0.0,0.0,0.0,1480.0,0.0
Worcester,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
